In [2]:
from pdg_js.build_pdg import get_data_flow
from pdg_js.js_operators import get_node_computed_value
import config as config 

def check_asyn_func_expr(node):
    for child in node.children:
        if child.name == 'IfStatement':
            print('find if statement')
        if child.name in ('CallExpression', 'TaggedTemplateExpression'):
            if len(child.children) > 0 and child.children[0].body in ('callee', 'tag'):
                callee = child.children[0]
                call_expr_value = get_node_computed_value(callee)
                if call_expr_value == 'wx.authorize' or call_expr_value in config.SENSITIVE_API:
                    pass

pdg_node = get_data_flow(input_file='testcase/case.js', benchmarks={}, save_path_pdg="testcase/case/pdg")
child = pdg_node.children[0].children[0]
obj_exp = child.children[1]
showCancel_value = True
for prop in obj_exp.children:
    if get_node_computed_value(prop.children[0]) == 'showCancel':
        showCancel_value = get_node_computed_value(prop.children[1])
    if get_node_computed_value(prop.children[0]) == 'success':
        scopes = check_asyn_func_expr(prop.children[1])

In [20]:
import os
import json

def handle_comp_vios_result_json():
    result_jsons = []
    compulsory_req = {}
    blanket_reqs = {}
    req_before_use = {}

    for root, dirs, files in os.walk('/root/minidroid/dataset/comp_vios'):
        for file in files:
            result_jsons.append(os.path.join(root, file))
    for result_json in result_jsons:
        with open(result_json, mode='r') as fp:
            data = json.load(fp)
            # result_correction
            for key1 in data.keys():
                if data[key1] != None:
                    for key2 in data[key1].keys():
                        data[key1][key2] = list(set(data[key1][key2]))
        fp.close()
        with open(result_json, mode='w+') as fp:
            json.dump(data, fp, indent=4)
        fp.close()

    for result_json in result_jsons:
        with open(result_json) as fp:
            data = json.load(fp)
            # result_filter
            if data['compulsory_req'] != None:
                compulsory_req[result_json] = data['compulsory_req']
            if data['req_before_use'] != None:
                req_before_use[result_json] = data['req_before_use']
            if data['blanket_reqs'] != None:
                blanket_reqs[result_json] = data['blanket_reqs']
    
    result = {
        'req_before_use': req_before_use,
        'blanket_reqs': blanket_reqs,
        'compulsory_req': compulsory_req
    }
    with open('/root/minidroid/dataset/comp_vios_result.json', 'w+') as fp:
        json.dump(result, fp, indent=4)

In [9]:
import os
import json

def handle_sensi_apis_result_json():
    result_jsons = []
    for root, dirs, files in os.walk('/root/minidroid/dataset/sensi_apis'):
        for file in files:
            result_jsons.append(os.path.join(root, file))
    results = {}
    for result_json in result_jsons:
        result = {}
        with open(result_json, mode='r') as fp:
            data = json.load(fp)
            if len(data):
                for key in data.keys():
                    for sensi_api in data[key]:
                        if sensi_api not in result.keys():
                            result[sensi_api] = list()
                            result[sensi_api].append(key)
                        else:
                            result[sensi_api].append(key)
                results[result_json] = result
    with open('/root/minidroid/dataset/sensi_apis_result.json', 'w+') as fp:
        json.dump(results, fp, indent=4)

handle_sensi_apis_result_json()

In [8]:
import json
import pprint
with open('/root/minidroid/dataset/sensi_apis_result.json', 'r', encoding='utf-8') as fp:
    data = json.load(fp=fp)
print(len(data))
apis = {}
page_num = 0
for key in data.keys():
    for api in data[key].keys():
        if api not in apis.keys():
            apis[api] = 1
            page_num+=len(data[key][api])
        elif api in apis.keys():
            apis[api]+=1
            page_num+=len(data[key][api])
# pprint.pprint(sorted(apis.items(), key = lambda kv:(kv[1], kv[0])))
print(page_num)

5920
67277
